In [1]:
# Add libraries 
import csv 
import os
import glob
import re 
from pandas import read_csv
import pandas as pd 
import numpy as np

In [2]:
path_all_input_datasets=r'C:\Users\shesa\Dropbox\Saagar\Sara\Code\Modified Code\Direct layer\input\2019_*.csv'
path_to_combine_dir=r"C:\Users\shesa\Dropbox\Saagar\Sara\Code\Modified Code\Direct layer\data preparation\data prep"
path_Combined_dataset=r"C:\Users\shesa\Dropbox\Saagar\Sara\Code\Modified Code\Direct layer\data preparation\Combined\2019_TL1.csv"
path_resultant_dataprep=r'C:\Users\shesa\Dropbox\Saagar\Sara\Code\Modified Code\Direct layer\data preparation\Combined\2019_TL1.csv'
path_filtered_dataset=r'C:\Users\shesa\Dropbox\Saagar\Sara\Code\Modified Code\Direct layer\data preparation\Filtered\Filtered.csv'
path_output_dataset=r'C:\Users\shesa\Dropbox\Saagar\Sara\Code\Modified Code\Direct layer\data preparation\Output\Final(Direct).csv'
path_sd_OP=r'C:\Users\shesa\Dropbox\Saagar\Sara\Code\Modified Code\Direct layer\sd\Combine\SD.csv'
path_sd_IP=r'C:\Users\shesa\Dropbox\Saagar\Sara\Code\Modified Code\Direct layer\sd\split'
FinalPath=r'C:\Users\shesa\Dropbox\Saagar\Sara\Code\Modified Code\Direct layer\OUTPUT(FINAL_DIRECT)\DirectLayer.csv'

In [3]:
#assign and drop headers and columns 
def Data_preparation(path):
    count=0
    for filename in glob.glob(path):
        Temp_df= pd.read_csv(filename)   
        Temp_df.drop(Temp_df.columns[len(Temp_df.columns)-1], axis=1, inplace=True)  
        Temp_df.drop(Temp_df.columns[len(Temp_df.columns)-1], axis=1, inplace=True)
        Temp_df.drop(Temp_df.columns[len(Temp_df.columns)-1], axis=1, inplace=True)
        Temp_df.columns = ['year','month (in digit)','DAY','DayType','Bin (min)','Shift (min)','Time (hr)','SampleSize','MeanTT(sec)','Median','Mode','Max','Min','Variance','StandardDev of TT (v2v)','LowerQUartile','UpperQuartile','95th_Percentile of TT','Buffer time','BufferTimeIndex'] # Assign headers to the dataset
        cols = ['year','month (in digit)','DAY']   
        newcol = ['-'.join(i) for i in Temp_df[cols].astype(str).values]
        Temp_df = Temp_df.assign(Date=newcol).drop(cols, 1)
        Temp_df["Date"]= pd.to_datetime(Temp_df["Date"])
        Temp_df["Day_of_week"]=Temp_df["Date"].dt.dayofweek
        Temp_df["Day_name"]=Temp_df["Date"].dt.day_name()
        Temp_df.to_csv(r'C:\Users\shesa\Dropbox\Saagar\Sara\Code\Modified Code\Direct layer\data preparation\data prep\C%s.csv' %count,index=False) # Save the modelled data for combination
        count=count+1 

#combine the prepared data into a single file         
def Data_combination(ipath,opath):
    os.chdir(ipath)
    extension = 'csv'
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
    combined_csv.to_csv(opath, index=False, encoding='utf-8-sig') 
    return combined_csv

#filter the day any time for the dataset 
def Data_filtering(ipath,opath):
    tl_df_de= pd.read_csv(ipath)
    Day_filter_de = tl_df_de[tl_df_de['DayType']==1]
    Time_filter1_de = Day_filter_de[Day_filter_de['Time (hr)']>=7]
    Time_filter2_de = Time_filter1_de[Time_filter1_de['Time (hr)']<=9]
    Time_filter2_de=Time_filter2_de.reset_index(drop=True)
    Time_filter2_de.to_csv(opath, index=False, encoding='utf-8-sig') 
    return Time_filter2_de

def sd(Filtered_df):
    Dow=Filtered_df['Day_of_week'].unique()
    Dow=list(Dow)
    Toh=Filtered_df['Time (hr)'].unique()
    Toh=list(Toh)
    count=0
    for x in Dow: 
        for y in Toh:
            df_8am=Filtered_df.loc[Filtered_df['Day_of_week']==x]
            df_8am=df_8am.loc[df_8am['Time (hr)']==y]
            df_8am=df_8am.reset_index(drop=True)
            df_8am['sample_Mean']=" "
            for i in range(len(df_8am)):
                df_8am.loc[i,'sample_Mean']=df_8am.loc[i,'MeanTT(sec)']*df_8am.loc[i,'SampleSize']
                        #x^=sum(x*n)/sum(n)
            df_8am['x']=df_8am['sample_Mean'].sum()/df_8am['SampleSize'].sum()    
            df_8am['d']=" "
            for i in range(len(df_8am)):
                df_8am.loc[i,'d']=df_8am.loc[i,'MeanTT(sec)']-df_8am.at[i,'x']         
            df_8am['variance_']=" "
            for i in range(len(df_8am)):
                df_8am.loc[i,'variance_']=df_8am.loc[i,'SampleSize']*(df_8am.loc[i,'Variance']+df_8am.loc[i,'d']**2)

            df_8am['S.D_at_Time']=np.sqrt(df_8am['variance_'].sum()/df_8am['SampleSize'].sum())
            df_8am.to_csv(r'C:\Users\shesa\Dropbox\Saagar\Sara\Code\Modified Code\Direct layer\sd\split\C%s.csv' %count,index=False)
            count=count+1
#Set a lower and upper bound for the mean travel time
def Time_bound(df):
    for i in range(len(df)):
        if (df.at[i,'MeanTT(sec)']<25):
            df.at[i,'MeanTT(sec)']=0
            df.at[i,'SampleSize']=0      
    for i in range(len(df)):
        if (df.at[i,'MeanTT(sec)']>300):
            df.at[i,'MeanTT(sec)']=0
            df.at[i,'SampleSize']=0
    
    return df 

#Calculate the standard error and weight for the dataset 
def Standard_error_and_weight(df,opath):
    df['StandardDEv(min)']=" " #Converting the SD into Minutes 
    for i in range(len(df)):
        df.at[i,'StandardDEv(min)']=(df.at[i,'S.D_at_Time']/60)

    df['standard error']=" " #Standard Error 
    for i in range(len(df)):
        df.at[i,'standard error']=df.at[i,'StandardDEv(min)']/np.sqrt((df.at[i,'SampleSize']))

    df['Weight']='' #Weight for Backward Layer     
    alpha=2.54
    e=2.71828
    for i in range(len(df)):
        df.at[i,'Weight']=alpha*(e**(-alpha*df.at[i,'standard error']))
    
    df.to_csv(opath, index=False, encoding='utf-8-sig')
    return df

In [9]:
if __name__ == '__main__' : 
    Data_preparation(path_all_input_datasets)
    dfc=Data_combination(path_to_combine_dir,path_Combined_dataset)
    Filtered_df=Data_filtering(path_resultant_dataprep,path_filtered_dataset)
    sd(Filtered_df)
    combined_sd=Data_combination(path_sd_IP,path_sd_OP)
    combined_sd=combined_sd.reset_index(drop=True)
    Bound_df=Time_bound(combined_sd)
#     Bound_df.drop(Bound_df.loc[Bound_df['SampleSize']==0].index,inplace=True)
#     Bound_df=Bound_df.reset_index(drop=True)
    OP=Standard_error_and_weight(Bound_df,path_output_dataset)
    OP=OP.sort_values(['Date', 'Time (hr)'], ascending=[True, True])
    OP=OP.reset_index(drop=True)
    OP.to_csv(FinalPath)

C:\Users\shesa\Anaconda3\lib\site-packages\ipykernel_launcher.py:85: RuntimeWarning: divide by zero encountered in double_scalars


In [10]:
OP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5929 entries, 0 to 5928
Data columns (total 28 columns):
DayType                    5929 non-null int64
Bin (min)                  5929 non-null int64
Shift (min)                5929 non-null int64
Time (hr)                  5929 non-null float64
SampleSize                 5929 non-null int64
MeanTT(sec)                5311 non-null float64
Median                     5311 non-null float64
Mode                       5311 non-null float64
Max                        5311 non-null float64
Min                        5311 non-null float64
Variance                   5311 non-null float64
StandardDev of TT (v2v)    5311 non-null float64
LowerQUartile              5311 non-null float64
UpperQuartile              5311 non-null float64
95th_Percentile of TT      5311 non-null float64
Buffer time                5311 non-null float64
BufferTimeIndex            5311 non-null float64
Date                       5929 non-null object
Day_of_week         